In [36]:
import transformers
from datasets import load_dataset, load_metric
from datasets.dataset_dict import DatasetDict
import datasets
import random
import pandas as pd
import numpy as np
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import argparse
import wandb
from typing import Tuple
import torch
from torch.nn import Module
from torch import Tensor
import torch.nn as nn
from transformers import PretrainedConfig
from transformers.models.t5.modeling_t5 import T5Stack, T5ForConditionalGeneration, T5Config
from transformers import T5Model
import sys
sys.path.append('../')
from datamodules.mt_distillation_datamodule import MTDistillationDatamodule

# model_checkpoint = "t5-small"
# model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

model = T5Model(config=T5Config.from_pretrained("t5-small"))
sentence = "translate English to Romanian: Hello, my dog is cute"
label = "Salut, câinele meu este frumos"
tokenizer = AutoTokenizer.from_pretrained("t5-small")
inputs = tokenizer(sentence, return_tensors="pt")
attention_mask = inputs["attention_mask"]
inputs = inputs["input_ids"]
decoder_input_ids = tokenizer(label, return_tensors="pt")
decoder_input_ids = decoder_input_ids["input_ids"]
decoder_attention_mask = decoder_input_ids.ne(tokenizer.pad_token_id).long()
outputs = model(input_ids=inputs, decoder_input_ids=decoder_input_ids, attention_mask=attention_mask, decoder_attention_mask=decoder_attention_mask)

logits = outputs[0]
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
loss = criterion(logits.permute(0, 2, 1), decoder_input_ids)
print(loss)
print(logits.shape)
print(logits)
print(tokenizer.decode(torch.argmax(logits, dim=-1).squeeze(0)))

/home/john/anaconda3/envs/dl4nlp/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


RuntimeError: Expected target size [1, 512], got [1, 9]

In [41]:
loss = criterion(logits.permute(0, 2, 1), decoder_input_ids)
logits.permute(0, 2, 1).shape

IndexError: Target 25801 is out of bounds.

In [9]:
from evaluate import load
perplexity = load("perplexity", module_type='metric')
dm = MTDistillationDatamodule(batch_size=8)
dm.setup()
first_batch = next(iter(dm.train_dataloader()))
# first_batch = {k: v.to(model.device) for k, v in first_batch.items()}
first_batch = first_batch['en-ro']
outputs = model.generate(input_ids=first_batch['input_ids'], attention_mask=first_batch['attention_mask'], max_length=128)
# predictions = {'predictions': predict, id=None)}
print(perplexity.compute(predictions=outputs, references=first_batch['decoder_input_ids']))

Using custom data configuration din0s--ccmatrix_en-ro-b5eb1c3b243f258f
Reusing dataset parquet (/home/john/.cache/huggingface/datasets/din0s___parquet/din0s--ccmatrix_en-ro-b5eb1c3b243f258f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /home/john/.cache/huggingface/datasets/din0s___parquet/din0s--ccmatrix_en-ro-b5eb1c3b243f258f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-246f20ddcdebc8fd.arrow
Using custom data configuration j0hngou--ccmatrix_en-fr-c75b778ef0fed3ea
Reusing dataset parquet (/home/john/.cache/huggingface/datasets/j0hngou___parquet/j0hngou--ccmatrix_en-fr-c75b778ef0fed3ea/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /home/john/.cache/huggingface/datasets/j0hngou___parquet/j0hngou--ccmatrix_en-fr-c75b778ef0fed3ea/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-7b6fd4d61823253e.arrow
Using custom data configuration j0hngou--ccmatrix_de-en-2acfbf58505ec757
Reusing dataset parquet (/home/john/.cache/huggingface/datasets/j0hngou___parquet/j0hngou--ccmatrix_de-en-2acfbf58505ec757/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /home/john/.cache/huggingface/datasets/j0hngou___parquet/j0hngou--ccmatrix_de-en-2acfbf58505ec757/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-cd6cc0e29ebdabbb.arrow
/home/john/anaconda3/envs/dl4nlp/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  wa

ValueError: Module inputs don't match the expected format.
Expected format: {'predictions': Value(dtype='string', id=None)},
Input predictions: tensor([[    0,  5636,     3,     9,  6344,   111, 20398,  7529,     3,    32,
           115, 11555,  3020,     6,     3,  7753,  1785, 15466,  1165,   142,
             3,   122, 26784,   111,  1746,   111,  1314,   630,     5,     1,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0],
        [    0,  9605,  4650,    15,  1456,    15,   176,     3, 10365,  6694,
           267,  5541,   185,   146,     7,    50,     3,    32, 21798,    98,
         21473,   111,   124, 21285,    20,  3748,   185, 23286,   111, 

In [11]:
first_batch

{'input_ids': tensor([[13959,  1566,    12,  3871,    29,    10,   938,   352,   441,    62,
             54,  1984,   959,     6,    38,    66,    13,     8,  4269,    33,
            435,   441,  3242,     5,     1,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0],
         [13959,  1566,    12,  3871,    29,    10,  9071, 15093,     7,    16,
              8,  6694,     7,    11,  5541,     7,  2237,    12,     3,     9,
           1827,  5362,    16,    84,  3986,  3977,    16,     8,  1336,  3738,
          17694,     3, 12884,     7,    13,  9975,     6,   192, 13090,  4897,
              7,     3,    26,    22,  7795,    16,  9047,   

In [69]:
def visualize_children(
    object,
    level : int = 0,
) -> None:
    """
    Prints the children of (object) and their children too, if there are any.
    Uses the current depth (level) to print things in a ordonnate manner.
    """
    print(f"{'   ' * level}{level}- {type(object).__name__}")
    try:
        for child in object.children():
            visualize_children(child, level + 1)
    except:
        pass

visualize_children(model)

0- T5ForConditionalGeneration
   1- Embedding
   1- T5Stack
      2- Embedding
      2- ModuleList
         3- T5Block
            4- ModuleList
               5- T5LayerSelfAttention
                  6- T5Attention
                     7- Linear
                     7- Linear
                     7- Linear
                     7- Linear
                     7- Embedding
                  6- T5LayerNorm
                  6- Dropout
               5- T5LayerFF
                  6- T5DenseActDense
                     7- Linear
                     7- Linear
                     7- Dropout
                     7- ReLU
                  6- T5LayerNorm
                  6- Dropout
         3- T5Block
            4- ModuleList
               5- T5LayerSelfAttention
                  6- T5Attention
                     7- Linear
                     7- Linear
                     7- Linear
                     7- Linear
                  6- T5LayerNorm
                  6- Dropout
         

In [79]:
from Distiller import Distiller

distiller = Distiller(model, 6)
student = distiller.get_student()

Model config PretrainedConfig {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "n_positions": 512,
  "num_decoder_layers": 2,
  "num_heads": 12,
  "num_layers": 2,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length": 300,
      "num_beams": 4,
      "prefix": "trans

In [80]:
student.num_parameters()

57714432